In [8]:
import os
print(os.listdir("/home/jovyan/data/delta_sncf_enriched"))


['.part-00000-027bd0d5-97cc-4c9a-b85e-7b4553a2ac80-c000.snappy.parquet.crc', '.part-00000-d3b322d3-9c7d-46be-ba33-1e82e97af736-c000.snappy.parquet.crc', '.part-00001-4210811a-24a0-4fac-a30a-97f65523eddd-c000.snappy.parquet.crc', '.part-00001-487d18f3-3ee2-4a59-bf1c-c1e7878cb258-c000.snappy.parquet.crc', '.part-00002-83af2db6-551b-4dd7-946f-24783c86a6db-c000.snappy.parquet.crc', '.part-00002-e6e7bea8-df45-4db2-a1dd-763659e38f71-c000.snappy.parquet.crc', '.part-00003-0f3ca7b8-f1a5-483a-87bd-7688020d1b28-c000.snappy.parquet.crc', '.part-00003-b17914fa-8070-4508-b1ef-3097318ec602-c000.snappy.parquet.crc', '.part-00004-43717c4c-7b47-4afc-88de-7fb67ff2019e-c000.snappy.parquet.crc', '.part-00004-57d2e603-49ef-4852-a95b-ec470d455071-c000.snappy.parquet.crc', '.part-00005-49bd73d8-fd51-4518-8e79-29b53ae7b61a-c000.snappy.parquet.crc', '.part-00005-efecb89e-b84a-4a26-83b7-13b31260ad0c-c000.snappy.parquet.crc', '.part-00006-38555449-d3b9-4637-8e25-ba8028c5cb73-c000.snappy.parquet.crc', '.part-0000

In [9]:
df = spark.read.parquet("/home/jovyan/data/delta_sncf_enriched")
df.write.option("header", True).mode("overwrite").csv("/home/jovyan/data/sncf_enriched_backup.csv")


In [10]:
df_trafic = spark.read.option("header", "true").option("inferSchema", "true") \
    .csv("/home/jovyan/data/sncf_enriched_backup.csv/")
df_trafic.show(5)


+--------------------+--------------------+----------+----------+----------+------------------+
|             stop_id|             trip_id|start_date|   arrival| departure|         stop_name|
+--------------------+--------------------+----------+----------+----------+------------------+
|StopPoint:OCEICE-...|OCESN9580F4261064...|  20250521|      NULL|1747828440|Paris Gare de Lyon|
|StopPoint:OCEICE-...|OCESN9580F4261064...|  20250521|1747831560|1747831680|             Melun|
|StopPoint:OCEICE-...|OCESN9580F4261064...|  20250521|1747832400|1747832520|     Fontainebleau|
|StopPoint:OCEICE-...|OCESN9580F4261064...|  20250521|1747834200|1747834320|         Montereau|
|StopPoint:OCEICE-...|OCESN9580F4261064...|  20250521|1747837200|1747837260|              Sens|
+--------------------+--------------------+----------+----------+----------+------------------+
only showing top 5 rows



In [13]:
df.write.option("header", True).mode("overwrite").csv("/home/jovyan/data/sncf_enriched_backup.csv")


In [15]:
from pyspark.sql.functions import col, from_unixtime, expr, round

# Créer une colonne datetime à partir du timestamp Unix
df_ts = df_trafic.withColumn("heure_arrivee", from_unixtime("arrival"))

# Simuler un retard de 5 à 30 minutes (aléatoire) pour illustrer le ML
df_ml = df_ts.withColumn("retard_arrivee", round(expr("rand() * 25 + 5")))  # entre 5 et 30 min

# Affichage des résultats
df_ml.select("stop_name", "arrival", "heure_arrivee", "retard_arrivee").show(5)


Py4JJavaError: An error occurred while calling o117.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 38) (0409e6268767 executor driver): org.apache.spark.SparkFileNotFoundException: File file:/home/jovyan/data/sncf_enriched_backup.csv/part-00000-77df44a9-0985-4c71-9c67-02278d101158-c000.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(Fi